<h1> Segmenting and Clustering Neighborhoods in Toronto

Import necessary libraries and read the Table from Wikipedia into a new dataframe

In [4]:
import pandas as pd

In [6]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df_html = pd.read_html(url)
df = df_html[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Check, if there are boroughs that have the value "not assigned" and drop these rows

In [7]:
df_bor = df[df.Borough != "Not assigned"].reset_index(drop = True)
df_bor.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Check the Neigbourhood, if there is any Neighbourhodd that is not assigned and replace it with the assigned borough. The replaced counter is only to check the replaced neighborhoods.

In [9]:
replaced = 0

for i in range(len(df_bor)):
    if df_bor["Borough"][i] != "Not assigned" and df_bor["Neighbourhood"][i] == "Not assigned":
        df_bor["Neighbourhood"][i].replace("Not assigned", df_bor["Borough"][i], inplace = True)
        replaced += 1
        
print(str(replaced))


0


Check, if more than one neighbourhood exists in one postal code area and group it

In [10]:
df_bor_grouped = df_bor.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)
df_bor_grouped.reset_index(inplace=True)

Look at the shape of the cleaned dataset

In [11]:
df_bor_grouped.shape

(103, 3)

Import geospatial data from website

In [2]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2020-08-02 11:35:53--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 158.85.108.83, 169.48.113.194, 158.85.108.86
Connecting to cocl.us (cocl.us)|158.85.108.83|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2020-08-02 11:35:53--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|158.85.108.83|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-02 11:35:56--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjj

Create a new Dataframe from csv

In [5]:
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two datasets and append the old dataset with coordinates

In [17]:
df_bor_geo = pd.merge(df_bor, df_geo, on = "Postal Code", how = "left")
df_bor_geo.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
